In [1]:
import ultralytics
from ultralytics import YOLO
import PIL
import os
import shutil
import albumentations as A
import cv2
import os
import numpy as np
from collections import Counter, defaultdict
import csv
import supervision as sv

ultralytics.checks()

Ultralytics 8.3.61 🚀 Python-3.12.8 torch-2.5.1+cu124 CUDA:0 (NVIDIA A30, 24167MiB)
Setup complete ✅ (96 CPUs, 502.8 GB RAM, 0.4/10.0 GB disk)


# Process and load data

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="y4b9MAg40VjykvXuH2Cv")
project = rf.workspace("malaria-classification").project("mal_dect_gametocyte_2")
version = project.version(13)
dataset = version.download("yolov8")
                
                
                
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to mal_dect_gametocyte_2-13 in yolov8:: 100%|██████████| 1426/1426 [00:05<00:00, 246.58it/s]


# Data augmentation

In [ ]:
# Directories
image_dir = 'C:/Users/mckay.m/Yolo_malaria_classification/mal_dect_gametocyte_2-2/train/images'
label_dir = 'C:/Users/mckay.m/Yolo_malaria_classification/mal_dect_gametocyte_2-2/train/labels'
augmented_image_dir = 'C:/Users/mckay.m/Yolo_malaria_classification/mal_dect_gametocyte_2-2/train_aug/images'
augmented_label_dir = 'C:/Users/mckay.m/Yolo_malaria_classification/mal_dect_gametocyte_2-2/train_aug/labels'

# Create directories to save augmented data if they don't exist
os.makedirs(augmented_image_dir, exist_ok=True)
os.makedirs(augmented_label_dir, exist_ok=True)

In [ ]:
# Define augmentation pipeline
#colourjitter
#gaussian blur
#guassian noise
# Set the seed
np.random.seed(42)
bbox_params = A.BboxParams(format='yolo', label_fields=['class_labels'])

transform = A.Compose([
    A.RandomCrop(width=640, height=640, p=0.5),        
    A.HorizontalFlip(p=0.5),                           
    #A.RandomBrightnessContrast(p=0.5),                 
    #A.HueSaturationValue(p=0.5),
    A.ColorJitter(p=0.5),                     
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.Blur(blur_limit=3, p=0.5),  
], bbox_params=bbox_params)

In [ ]:
# Function to load YOLO annotations
def load_yolo_labels(label_path):
    bboxes = []
    class_labels = []
    with open(label_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"Skipping invalid line in file: {label_path}")
                continue
            class_id, x_center, y_center, width, height = map(float, parts)
            # Check if values are within valid range
            if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                print(f"Invalid bbox values found in file: {label_path}")
                continue
            bboxes.append([x_center, y_center, width, height])
            class_labels.append(int(class_id))
    return bboxes, class_labels

def save_yolo_labels(label_path, bboxes, class_labels):
    with open(label_path, 'w') as file:
        for bbox, class_label in zip(bboxes, class_labels):
            file.write(f"{class_label} {' '.join(map(str, bbox))}\n")

In [ ]:
# Loop through all images and labels
for image_filename in os.listdir(image_dir):
    if image_filename.endswith('.jpg'):
        # Load image
        image_path = os.path.join(image_dir, image_filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
        height, width, _ = image.shape

        # Get corresponding label file
        label_filename = image_filename.replace('.jpg', '.txt')
        label_path = os.path.join(label_dir, label_filename)

        # Process label file if it exists
        if os.path.exists(label_path):
            bboxes, class_labels = load_yolo_labels(label_path)

            # Skip images with invalid bounding boxes
            if not bboxes:
                print(f"No valid bounding boxes found in label file: {label_filename}")
                continue
            
            # Apply augmentations
        try:

            for i in range(1):  # Generate 3 augmented images per original image
                augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                augmented_image = augmented['image']
                augmented_bboxes = augmented['bboxes']
                augmented_labels = augmented['class_labels']
                
                # Save augmented image
                augmented_image_filename = f"{image_filename.replace('.jpg', f'_aug_{i}.jpg')}"
                augmented_image_path = os.path.join(augmented_image_dir, augmented_image_filename)
                cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
                
                # Save augmented labels
                augmented_label_filename = f"{label_filename.replace('.txt', f'_aug_{i}.txt')}"
                augmented_label_path = os.path.join(augmented_label_dir, augmented_label_filename)
                save_yolo_labels(augmented_label_path, augmented_bboxes, augmented_labels)
        except ValueError as e:
                print(f"Error processing file {image_filename}: {e}")

print("Augmentation complete. Augmented images and labels saved.")

# Train model

In [3]:
#load pretrained model 
model = YOLO("yolov8x.pt")

In [4]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Displays your GPU name

True
NVIDIA A30


In [5]:
# Train the model using the 'data_1.yaml' dataset for 100 epochs
#to test: single_cls,cos_lr, lr0, lrf, dropout
results = model.train(data="/vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-13/data.yaml", epochs=100, patience=50, device=0)

Ultralytics 8.3.61 🚀 Python-3.12.8 torch-2.5.1+cu124 CUDA:0 (NVIDIA A30, 24167MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-13/data.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

train: Scanning /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-13/train/labels... 568 images, 11 backgrounds, 0 corrupt: 100%|██████████| 568/568 [00:00<00:00, 2117.90it/s]

train: New cache created: /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-13/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



Argument 'quality_lower' is not valid and will be ignored.
This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
val: Scanning /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-13/valid/labels... 93 images, 6 backgrounds, 0 corrupt: 100%|██████████| 93/93 [00:00<00:00, 1546.27it/s]

val: New cache created: /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-13/valid/labels.cache


...

     86/100      13.2G     0.6026     0.3501     0.8469         42        640: 100%|██████████| 36/36 [00:13<00:00,  2.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.30it/s]

                   all         93        523      0.732      0.695      0.775      0.417
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 36, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



86 epochs completed in 0.413 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train/weights/best.pt, 136.7MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.61 🚀 Python-3.12.8 torch-2.5.1+cu124 CUDA:0 (NVIDIA A30, 24167MiB)
Model summary (fused): 268 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.62it/s]


                   all         93        523      0.682      0.772      0.805      0.425
               Stage-2         20        140      0.638      0.786      0.762      0.343
               Stage-3         43        205      0.828      0.815       0.89      0.496
               Stage-4         31         58       0.51      0.603       0.67       0.36
               Stage-5         39        120      0.751      0.883      0.897      0.502
Speed: 0.5ms preprocess, 6.4ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/train


# Create confidence scores csv and detection counts csv

In [5]:
# Load the pre-trained YOLO model
model = YOLO('/vast/scratch/users/mckay.m/malaria_classifier_katrina/runs/detect/train_2_11_full_images_size_640_best_so_far/weights/best.pt')

img_dir = '/vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-11/test/images'

# Run predictions on the test set images
results = model.predict(img_dir, save=True)

# Initialize data structures
all_stages = set()
csv_data = []
confidence_data = []

# Loop through results to find all unique stages
for result in results:
    # Extract the image name after the last "/"
    image_name = os.path.basename(result.path)
    
    # Get detected class indices and corresponding confidence scores
    class_indices = result.boxes.cls.int().tolist()
    confidences = result.boxes.conf.tolist()
    
    # Map class indices to stage names and create a list of tuples (stage, confidence)
    detected = [(model.names[class_idx], conf) for class_idx, conf in zip(class_indices, confidences)]
    
    # Add detected stages to the all_stages set
    all_stages.update(stage for stage, _ in detected)

# Convert the set of all stages to a sorted list
all_stages = sorted(all_stages)

# Totals for each stage
total_counts = Counter()

# Loop through results again to build rows for both CSVs
for result in results:
    # Extract the image name
    image_name = os.path.basename(result.path)
    
    # Get detected class indices and corresponding confidence scores
    class_indices = result.boxes.cls.int().tolist()
    confidences = result.boxes.conf.tolist()
    
    # Map class indices to stage names and create a list of tuples (stage, confidence)
    detected = [(model.names[class_idx], conf) for class_idx, conf in zip(class_indices, confidences)]
    
    # Count the occurrences of each detected stage
    stage_count = Counter(stage for stage, _ in detected)
    
    # Update totals for stage counts
    total_counts.update(stage_count)
    
    # Create a row for the detection counts
    count_row = [image_name]
    for stage in all_stages:
        count_row.append(stage_count.get(stage, 0))
    csv_data.append(count_row)
    
    # Create a row for confidence scores as a single column
    confidence_list = [f"{stage}:{round(conf, 2)}" for stage, conf in detected]
    confidence_row = [image_name, ", ".join(confidence_list)]
    confidence_data.append(confidence_row)

# Add totals to the counts CSV
total_row_counts = ["Total"]
for stage in all_stages:
    total_row_counts.append(total_counts.get(stage, 0))
csv_data.append(total_row_counts)

# Write the detection counts to the first CSV file
csv_file_counts = 'detection_results_counts.csv'
with open(csv_file_counts, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name"] + all_stages)
    writer.writerows(csv_data)

print(f"Detection counts saved to {csv_file_counts}")

# Write the confidence scores to the second CSV file
csv_file_confidences = 'detection_results_confidences.csv'
with open(csv_file_confidences, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name", "Detections with Confidence"])
    writer.writerows(confidence_data)

print(f"Confidence scores saved to {csv_file_confidences}")



image 1/46 /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-11/test/images/day10_WT-21_png.rf.f77866c4e9ce7cf5cfd9682b8ce38c36.jpg: 640x480 1 Stage-4, 4 Stage-5s, 43.8ms
image 2/46 /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-11/test/images/day10_WT-27_png.rf.aff56612626071d1d34f1ef52bdfc15d.jpg: 640x480 1 Stage-4, 1 irregular, 18.7ms
image 3/46 /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-11/test/images/day10_WT-28_png.rf.6f1ced73a46fa4d0f81834e4256b90d1.jpg: 640x480 1 irregular, 18.7ms
image 4/46 /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-11/test/images/day10_WT-33_png.rf.417787af3e4e77acc5554193a862d50f.jpg: 640x480 1 Stage-5, 18.7ms
image 5/46 /vast/scratch/users/mckay.m/malaria_classifier_katrina/mal_dect_gametocyte_2-11/test/images/day10_WT-67_png.rf.bc1df8b762101d0c47ea0b2179a8af1b.jpg: 640x480 1 Stage-5, 18.7ms
image 6/46 /vast/scratch/users/mckay.m/mal